In [ ]:
import os 
os.chdir('..')

In [ ]:
import pandas as pd

In [ ]:
from pathlib import Path
import numpy as np 

In [ ]:
df = pd.read_pickle("artifact/stage02_data_validation/validated_test.pkl")

In [ ]:
x = df.drop(columns=["default" , "ID"])

In [ ]:
from CreditCard.utils import load_object , read_yaml 

In [ ]:
preproccesin_obj = load_object(Path("/home/pk/project-python/creditcard/artifact/stage03_data_transformation/preprocessing/preprocessing_obj.pkl"))

In [ ]:
x_transformed = preproccesin_obj.transform(x)

In [ ]:
columns = preproccesin_obj.encoder.get_feature_names_out().tolist()

In [ ]:
columns.append("cluster")

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
y = df['default']

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split( x_transformed, y.values, test_size=0.33, random_state=42)

In [ ]:
X_train.shape[1]

In [ ]:
type(X_train)

In [ ]:
type(y_train)

In [ ]:
model = CatBoostClassifier(iterations=2, learning_rate=1, depth=2, loss_function='Logloss', verbose=True)

model.fit(X_train, y_train)

In [ ]:
model_best = evaluate_classification_model(x_train_eval=X_train, y_train=y_train, x_test_eval=X_test, y_test=y_test, base_accuracy=0.5, report_dir=".", 
                                           eval_difference=0.05, estimators=[model] , columns = columns)

In [ ]:
from CreditCard.utils import read_yaml_as_dict , write_yaml , save_numpy_array_data
import numpy as np

In [ ]:
data = read_yaml_as_dict(Path('/home/pk/project-python/creditcard/configs/schema.yaml'))

In [ ]:
data_to_update = {"base_model_trained_columns"  : columns}

In [ ]:
data.update(data_to_update)

In [ ]:
write_yaml(data= data, file_path=Path('/home/pk/project-python/creditcard/configs/schema.yaml'))

In [ ]:
x_transformed

In [ ]:
data_save_path = os.path.join("/home/pk/project-python/creditcard/artifact/stage03_optuna_training" , "train.npz")

In [ ]:
data_to_save = np.c_[x_transformed, y.values]

In [ ]:
type(data_to_save)

In [ ]:
save_numpy_array_data(file_path=Path(data_save_path),  array=data_to_save)

In [ ]:
from CreditCard.utils import load_numpy_array_data 

In [ ]:
arr = load_numpy_array_data(Path(data_save_path))

In [ ]:
from CreditCard.model_factory import ModelFactory

In [ ]:
model_factor = ModelFactory(model_factory_config_path=Path("configs/model_factoryoptuna.yaml") , data_to_train=arr)

In [ ]:
model_report_path = model_factor.initiate_model_params_search()

In [ ]:
model_list = model_factor.get_best_model(model_report_path= model_report_path)

In [ ]:
from CreditCard.utils import  read_yaml
schema = read_yaml(Path("configs/schema.yaml"))

In [ ]:
columns_ = schema.base_model_trained_columns

In [ ]:
best_model = model_factor.get_best_evaluated_model(train_model_list = model_list , base_accuracy =0.6, base_report_dir  = os.path.dirname( model_report_path), 
                                 eval_difference = 0.5, eval_param = "accuracy",columns_trained_on = columns_)

In [ ]:
import os 
os.chdir('..')

In [ ]:

import pandas as pd
from pathlib import Path
import numpy as np 
data_save_path = os.path.join("/home/pk/project-python/creditcard/artifact/stage03_optuna_training" , "train.npz")


from CreditCard.utils import load_numpy_array_data 
from CreditCard.model_factory import ModelFactory
arr = load_numpy_array_data(Path(data_save_path))
model_factor = ModelFactory(model_factory_config_path=Path("configs/model_factoryoptuna.yaml") , data_to_train=arr)
model_report_path = model_factor.initiate_model_params_search()
model_list = model_factor.get_best_model(model_report_path= model_report_path)
from CreditCard.utils import  read_yaml
schema = read_yaml(Path("configs/schema.yaml"))
columns_ = schema.base_model_trained_columns
best_model = model_factor.get_best_evaluated_model(train_model_list = model_list , base_accuracy =0.6, base_report_dir  = os.path.dirname( model_report_path), 
                                 eval_difference = 0.5, eval_param = "accuracy",columns_trained_on = columns_)



In [ ]:
from xgboost import XGBClassifier

In [ ]:
x = arr[:,:-1]

In [ ]:
y = arr[:,-1]

In [ ]:
y = y.astype(int)

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( x, y, test_size=0.33, random_state=42)

In [ ]:
model = XGBClassifier()

model.fit(X_train, y_train)

In [ ]:
model.predict_proba(X_test)[1]

In [ ]:
for model in model_list:
    print(model.predict_proba(X_test)[1])

In [1]:
from CreditCard.utils import load_object , read_yaml , read_yaml_as_dict , write_yaml
from pathlib import Path
import os 

os.chdir('..')

In [2]:
pre = load_object(Path("/home/pk/project-python/creditcard/artifact/stage03_data_transformation/preprocessing/preprocessing_obj.pkl"))

In [7]:
data =  {col: int for col in pre.encoder.feature_names_in_}

In [8]:
data

{'LIMIT_BAL': int,
 'SEX': int,
 'EDUCATION': int,
 'MARRIAGE': int,
 'AGE': int,
 'PAY_0': int,
 'PAY_2': int,
 'PAY_3': int,
 'PAY_4': int,
 'PAY_5': int,
 'PAY_6': int,
 'BILL_AMT1': int,
 'BILL_AMT2': int,
 'BILL_AMT3': int,
 'BILL_AMT4': int,
 'BILL_AMT5': int,
 'BILL_AMT6': int,
 'PAY_AMT1': int,
 'PAY_AMT2': int,
 'PAY_AMT3': int,
 'PAY_AMT4': int,
 'PAY_AMT5': int,
 'PAY_AMT6': int}

In [4]:
schema = read_yaml_as_dict(Path("configs/schema.yaml"))

2023-03-29 09:26:12.390 | INFO     | CreditCard.utils.common:read_yaml_as_dict:58 - yaml file: configs/schema.yaml loaded successfully


In [9]:
schema["columns_to_eval"] = pre.encoder.feature_names_in_.tolist()

In [10]:
schema["columns_to_eval"]

['LIMIT_BAL',
 'SEX',
 'EDUCATION',
 'MARRIAGE',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

In [11]:
write_yaml(data= schema, file_path=Path("configs/schema.yaml"))

In [5]:
from CreditCard.utils import write_json
import json
from pathlib import Path
import os 

In [6]:
data = {"ssss" : 1222}

In [7]:
file_path = os.path.join(os.getcwd() , "test.json")

In [9]:
write_json(data_to_dump=data , file_path= Path(file_path))

2023-03-29 11:47:03.324 | INFO     | CreditCard.utils.common:write_json:242 - writing json file at: /home/pk/project-python/creditcard/research/test.json


In [3]:
from CreditCard.utils import load_object
from pathlib import Path 

In [4]:
model = load_object(Path("/home/pk/project-python/creditcard/artifact/stage04_model_evaluation/evaluated_model_dir/evaluated_model.pkl"))

In [5]:
params = model.trained_model_object.get_params()

In [6]:
# !pip install dvclive

In [7]:
from dvclive import Live

In [9]:
Live.log_param?

Signature:
Live.log_param(
    self,
    name: str,
    val: Union[int, float, str, bool, List[ForwardRef('ParamLike')], Dict[str, ForwardRef('ParamLike')]],
)
Docstring: Saves the given parameter value to yaml
File:      ~/miniconda3/envs/credit_card/lib/python3.9/site-packages/dvclive/live.py
Type:      function

In [10]:
with Live() as live:
    
    for param , value in params.items():
        live.log_param(name= param , val=value)

INFO:dvclive:To run with DVC, add this to /home/pk/project-python/creditcard/dvc.yaml:
stages:
  dvclive:
    cmd: <python my_code_file.py my_args>
    deps:
    - <my_code_file.py>



In [17]:
from CreditCard.utils import load_object , read_yaml_as_dict , write_yaml
from pathlib import Path

In [18]:
pre = load_object(Path("/home/pk/project-python/creditcard/artifact/stage03_data_transformation/preprocessing/preprocessing_obj.pkl"))

In [19]:
len(pre.encoder.get_feature_names_out().tolist())

106

In [20]:
schema = read_yaml_as_dict(Path("/home/pk/project-python/creditcard/configs/schema.yaml"))

2023-03-31 07:28:58.494 | INFO     | CreditCard.utils.common:read_yaml_as_dict:61 - yaml file: /home/pk/project-python/creditcard/configs/schema.yaml loaded successfully


In [21]:
schema["base_model_trained_columns"] = pre.encoder.get_feature_names_out().tolist()

In [22]:
write_yaml(data= schema, file_path=Path("/home/pk/project-python/creditcard/configs/schema.yaml"))

In [1]:
from CreditCard.utils import load_object
from pathlib import Path 